In [25]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
import numpy as np
import string
import re

Criação do conjunto de dados

In [26]:
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file,sep='\t')
        self.word_to_idx = {}
        self.idx_to_word = {}
        self.build_vocab()

    def build_vocab(self):
        words = ' '.join(self.data['text']).lower()
        words = re.sub('['+string.punctuation+']', '', words)
        words = words.split()
        unique_words = set(words)
        self.word_to_idx = {word: idx for idx, word in enumerate(unique_words)}
        self.idx_to_word = {idx: word for word, idx in self.word_to_idx.items()}
        novo_idx = len(self.word_to_idx)
        self.word_to_idx['<END>'] = novo_idx
        self.idx_to_word[novo_idx] = '<END>'

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['text']
        text = text.lower()
        text = re.sub('['+string.punctuation+']', '', text)

        tokens = [token for token in text.split() if token != '']
        labels = [tokens[i+1] if i < len(tokens)-1 else '<END>' for i in range(len(tokens))]

        input_ids = [self.word_to_idx[token] for token in tokens]
        label_ids = [self.word_to_idx[label] for label in labels]
        return {'input_ids': torch.tensor(input_ids), 'labels': torch.tensor(label_ids)}

Criação da classe relacionada ao modelo RNN

In [27]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim

    def forward(self, x):
        x = self.embedding(x)
        output, hidden = self.rnn(x)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_dim)

Hiperparâmetros

In [28]:
embedding_dim = 100
hidden_dim = 256
num_layers = 2
learning_rate = 0.0001
num_epochs = 50

Carrega o conjunto de dados

In [29]:
dataset = CustomDataset('https://raw.githubusercontent.com/giacicunb/enap_pln2024/main/corpora/simple_corpus.csv')
dataloader = DataLoader(dataset, batch_size=1)

Verificando se está tudo correto com nosso dataset

Obtém-se o tamanho do vocabulário

In [30]:
vocab_size = len(dataset.word_to_idx)
print(f'O vocaculario possui {vocab_size} palavras')

O vocaculario possui 362 palavras
O vocaculario possui 362 palavras


Instanciando o modelo baseado em RNN

In [31]:
model = RNNModel(vocab_size+1, embedding_dim, hidden_dim, num_layers)

 Definindo a função de perda, em que a função softmax está implementada internamente:

In [32]:
loss_function = nn.CrossEntropyLoss()

Definindo-se o otimizador Adam

In [33]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Treinamento da RNN



1.   O treinamento é feito manualmente, pegando-se batch por batch
2.   Primeiramente, pega-se os textos do batch sendo analisados e prepara-os para colocá-los no modelo
3.   Inicializa $h^{(0)}$
4.   Depois, calcula $h^{(t)}$
5.   Após processar todas as palavras do texto de entrada, calcula-se a função loss
6.   Atualiza os pesos da rede fazendo-se o backpropagation

In [22]:
for epoch in range(num_epochs):

    total_loss = 0
    for batch in dataloader:

        tokens, labels = batch['input_ids'], batch['labels']

        optimizer.zero_grad()

        output,_ = model(tokens)

        output_flat = output.view(-1, output.shape[-1])
        labels_flat = labels.view(-1)

        loss = loss_function(output_flat, labels_flat)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}')

Epoch 1, Loss: 5.902366638183594
Epoch 2, Loss: 5.850257555643718
Epoch 3, Loss: 5.80612850189209
Epoch 4, Loss: 5.762238105138143
Epoch 5, Loss: 5.717470089594523
Epoch 6, Loss: 5.6708824634552
Epoch 7, Loss: 5.621278524398804
Epoch 8, Loss: 5.566962560017903
Epoch 9, Loss: 5.5057750542958575
Epoch 10, Loss: 5.4366841316223145
Epoch 11, Loss: 5.364171981811523
Epoch 12, Loss: 5.2970796426137285
Epoch 13, Loss: 5.2382363478342695
Epoch 14, Loss: 5.1847489674886065
Epoch 15, Loss: 5.13417402903239
Epoch 16, Loss: 5.0859731038411455
Epoch 17, Loss: 5.040077288945516
Epoch 18, Loss: 4.99564266204834
Epoch 19, Loss: 4.951560417811076
Epoch 20, Loss: 4.907270431518555
Epoch 21, Loss: 4.862528006235759
Epoch 22, Loss: 4.8171153863271075
Epoch 23, Loss: 4.77091646194458
Epoch 24, Loss: 4.72383451461792
Epoch 25, Loss: 4.675752401351929
Epoch 26, Loss: 4.626632928848267
Epoch 27, Loss: 4.576525529225667
Epoch 28, Loss: 4.5255117019017534
Epoch 29, Loss: 4.473669449488322
Epoch 30, Loss: 4.4210

Predição

In [34]:
def predict_next_word(model, input_text):
    input_text = input_text.split()
    input_data = torch.tensor([[dataset.word_to_idx[word] for word in input_text]])
    output, _ = model(input_data)
    _, predicted_idx = torch.max(output[:, -1], 1)
    predicted_word = dataset.idx_to_word[predicted_idx.item()]
    return predicted_word

Fazendo o modelo de linguagem funcionar

In [40]:
input_text = "termo"
predicted_word = predict_next_word(model, input_text)
print(f"A proxima palavra apos {input_text} eh {predicted_word}")

A proxima palavra apos termo eh como
